# Introduction

1. I might be doing the scalar multiplies with the transpose -- if things don't work, check here
2. Check the actor loss -- am I screwing things up with the tranposing?

3. Implement prioritized experience replay.


### Test on cartpole

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from agent import Agent
%matplotlib inline


#Environment
env = gym.make('CartPole-v0')
env.seed(0)
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n


#Agent
lr,gamma,tau = 3*10**-4, 0.99, 0.01, 
clipnorm, verbose, alpha = False, False, 0.001
agent = Agent(input_dim, output_dim, lr, gamma, tau, alpha, clipnorm, verbose)
agent.memory_size = 2000
agent.batchsize = 32


#Train
EPISODES = 10**4
scores = []
for e in range(1,EPISODES+1):
    state = env.reset()
    state = agent.make_tensor(state)
    reward_sum = 0
    done = False
    while not done:
        
        #Do main step
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward_sum += reward
        next_state = agent.make_tensor(next_state)
        agent.remember(state[0],action,reward,next_state[0],done) #want to remember state as a vec
        state = next_state

        #Learn
        if e > 50:
            agent.learn()
    
    #Print results
    scores.append(reward_sum)
    if e % 50 == 0:
        print '(episode, score) = ' + str((e,reward_sum))
plt.plot(scores)

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(episode, score) = (50, 17.0)
(episode, score) = (100, 15.0)
(episode, score) = (150, 17.0)
(episode, score) = (200, 26.0)
(episode, score) = (250, 33.0)
(episode, score) = (300, 10.0)
(episode, score) = (350, 11.0)
(episode, score) = (400, 10.0)
(episode, score) = (450, 11.0)
(episode, score) = (500, 10.0)
(episode, score) = (550, 10.0)
(episode, score) = (600, 10.0)
(episode, score) = (650, 9.0)
(episode, score) = (700, 10.0)
(episode, score) = (750, 8.0)
(episode, score) = (800, 9.0)
(episode, score) = (850, 8.0)
(episode, score) = (900, 9.0)
(episode, score) = (950, 10.0)
(episode, score) = (1000, 9.0)
(episode, score) = (1050, 10.0)
(episode, score) = (1100, 11.0)
(episode, score) = (1150, 9.0)
(episode, score) = (1200, 9.0)
(episode, score) = (1250, 9.0)
(episode, score) = (1300, 8.0)
(episode, score) = (1350, 9.0)
(episode, score) = (1400, 9.0)
(episode, score) = (1450, 11.0)
(episo

KeyboardInterrupt: 

### Debug the learning

In [2]:
#get batch
S2,A2,R2,S12,D2 = agent.get_batch()

#Get what we need for learning
Q2 = agent.criticQ.model.predict([S2,A2])
V2 = agent.criticV.target_model.predict(S12)  #value of NEXT state

Pi_vec = agent.actor.model.predict(S2)
Pi2 = np.sum(Pi_vec*A2,axis=1)          #now a batch of \pi(s_t,a_t)
Pi2 = np.array([[i] for i in Pi2])

#train actor
agent.actor.learn(S2,A2,Q2,V2)

#train critic
agent.criticQ.learn(S2,A2,R2,V2)

#train value funtion
agent.criticV.learn(S2,Q2,Pi2)

ValueError: setting an array element with a sequence.

In [4]:
indicies = np.random.choice(range(len(agent.S)),agent.batchsize)
S = np.array(agent.S)[indicies]
A = np.array(agent.A)[indicies]
R = np.array(agent.R)[indicies]
S1 = np.array(agent.S1)[indicies]
D = np.array(agent.D)[indicies]

ValueError: setting an array element with a sequence.

In [9]:
agent.S

[array([-0.06510387, -0.92482313,  0.04794288,  1.46648494]),
 array([-0.04363321, -0.14877061,  0.01284913,  0.2758415 ]),
 array([-0.04660862,  0.04616568,  0.01836596, -0.01276126]),
 array([-0.04568531, -0.14921478,  0.01811073,  0.28565929]),
 array([-0.04866961, -0.34459028,  0.02382392,  0.58399874]),
 array([-0.05556141, -0.54003773,  0.03550389,  0.88409029]),
 array([-0.06636217, -0.3454154 ,  0.0531857 ,  0.60277673]),
 array([-0.07327047, -0.54123934,  0.06524124,  0.91172651]),
 array([-0.08409526, -0.73718048,  0.08347577,  1.22418026]),
 array([-0.09883887, -0.5432269 ,  0.10795937,  0.95877631]),
 array([-0.10970341, -0.349709  ,  0.1271349 ,  0.70186788]),
 array([-0.11669759, -0.15655704,  0.14117225,  0.45175531]),
 array([-0.11982873,  0.03631571,  0.15020736,  0.20669124]),
 array([-0.11910242,  0.22900623,  0.15434119, -0.03509494]),
 array([-0.11452229,  0.42161681,  0.15363929, -0.2753801 ]),
 array([-0.10608995,  0.22467477,  0.14813168,  0.06154565]),
 array([

### CriticV 

In [46]:
#Define model
S = Input(shape=(input_dim,))
x = Dense(128, activation = 'relu')(S)
out = Dense(1, activation = 'linear')(x)
model = Model(inputs = S, outputs = out)

#Find terms in bracket
S_pl = model.input
Pi_pl = K.placeholder(shape=(None,1))
Q_pl = K.placeholder(shape=(None,1))
V_pl = model.output
#temp = V_pl - Q_pl + self.alpha*K.log(Pi_pl)

#Find gradient
pars = model.trainable_weights
grads = tf.gradients(V_pl, pars, V_pl - Q_pl + Pi_pl)  #scalar multiply by temp

#Do learning
#To get keras to apply updates given a custom gradients (i.e. run the above line) I had to alter the source
#Code. It was easy to do. See line X in the get_updates function.
#opt = Adam(self.lr)
#loss = grads  #placeholder, keras doesn't use it
#updates = opt.get_updates(loss = loss, params = pars, grads = grads)

#This function will apply updates when called
func = K.function(inputs = [S_pl, Q_pl, Pi_pl], outputs = [x for x in grads])
func([S2,Q2,Pi3])

[array([[ 1.51027110e-03, -1.45281553e-02, -1.27284112e-03,
         -1.99574931e-03,  0.00000000e+00,  2.11893637e-02,
          3.41017591e-03, -3.24681103e-02, -2.23310571e-02,
         -1.75426975e-02, -6.00003498e-03,  2.85510365e-02,
          2.99270195e-03,  1.86458963e-03, -1.27616641e-03,
         -1.37190530e-02,  1.12744514e-03, -1.00985505e-02,
          1.83073822e-02,  2.29555015e-02, -1.54627813e-03,
          3.32161374e-02,  1.62787270e-02, -2.91726906e-02,
          1.10129907e-03,  2.91941091e-02, -2.91204336e-03,
          6.46402710e-04,  3.11365281e-03, -1.24060910e-03,
          2.09052479e-04,  1.67605653e-02,  1.21758655e-02,
         -2.34417580e-02,  3.56843323e-03, -1.24385357e-02,
          2.86449143e-03, -2.12741550e-02, -2.21519498e-03,
          3.40363849e-03, -1.35363685e-03, -4.56820009e-03,
         -2.57358328e-03, -4.02094517e-03,  3.64861684e-03,
         -2.33322866e-02,  1.65733771e-04, -3.14151086e-02,
          2.39921384e-03, -1.95071660e-0

In [35]:
Pi3 = np.array([[i] for i in Pi2])
Pi3

array([[0.5052697 ],
       [0.4708679 ],
       [0.45295146]], dtype=float32)

In [40]:
V2 - Q2 + alpha*np.log(Pi3)

array([[-0.02717253],
       [ 0.17803136],
       [-0.01151956]], dtype=float32)

In [45]:
V2.shape, Q2.shape, Pi3.shape

((3, 1), (3, 1), (3, 1))

In [48]:
Pi2.shape == V2.shape

False

In [ ]:
as

### Critic network

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam 
from keras.optimizers import RMSprop
from keras import backend as K
from keras.utils import to_categorical
from keras.layers import Dense, Input, concatenate

input_dim, output_dim = 4,2
alpha = 0.1
lr = 0.1
gamma = 0.9

S = Input(shape=(input_dim,))
A = Input(shape=(output_dim,))
x = concatenate([S,A])
x = Dense(5, activation = 'relu')(x)
out = Dense(1, activation = 'linear')(x)
model = Model(inputs = [S,A], outputs = out)

#Input
S_pl, A_pl = model.input
Q_pl = model.output

#Find term in bracket
V_pl = K.placeholder(shape=(None,1))
R_pl = K.placeholder(shape=(None,1)) 
temp = Q_pl - R_pl + gamma*V_pl

#Find gradient
pars = model.trainable_weights
grads = tf.gradients(Q_pl,pars,temp)  #scalar multiply by temp

#Do update
opt = Adam(lr)
loss = grads
updates = opt.get_updates(loss = loss, params = pars, grads = grads)


func = K.function(inputs=[S_pl,A_pl,R_pl,V_pl],outputs=[x for x in grads], updates = updates)
func([S2,A2,R2,V2])

[array([[ 0.04806015,  0.15712519, -0.14912567,  0.05254675,  0.        ],
        [ 0.06267454,  0.9315323 , -1.1589466 ,  0.06852545,  0.        ],
        [-0.0440952 , -0.06870569,  0.03666694, -0.04821166,  0.        ],
        [-0.03239084, -1.1025333 ,  1.4233701 , -0.03541465,  0.        ],
        [ 1.2114152 , -1.7570772 ,  3.8302555 ,  1.3245054 ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]],
       dtype=float32),
 array([ 1.2114152, -1.7570772,  3.8302555,  1.3245054,  0.       ],
       dtype=float32),
 array([[-0.12519524],
        [-0.5022066 ],
        [-1.7734098 ],
        [-0.03578898],
        [ 0.        ]], dtype=float32),
 array([-4.0047693], dtype=float32)]

### Actor network

In [16]:
import tensorflow as tf
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


#Inputs
state_pl = model.input
action_pl = K.placeholder(shape=(None, output_dim))  #onehot
Q_pl = K.placeholder(shape=(None,1))
V_pl = K.placeholder(shape=(None,1))

#Find terms in bracket
pi_vec = model.output
pi = K.sum(pi_vec*action_pl,axis=1)    # get \pi(s_t, a_t) -- prob for specific action
entropy = alpha*K.log(pi)
temp = entropy - K.transpose(Q_pl) + K.transpose(V_pl)  #this is a row vec
temp = K.transpose(temp)      #turn it into col vec

#Find grad log(pi)
pi_pl = model.output
pars = model.trainable_weights
grads = tf.gradients( K.log(pi_pl), pars, temp)   #scalar multiply by temp

#Do learning
opt = Adam(lr)
loss = grads  #placeholder, I won't use it
updates = opt.get_updates(loss = grads, params = pars, grads = grads)

# To get keras to apply updates given a custom gradients (i.e. run the above line) I had to alter the source
# Code. It was easy to do. See line X in the get_updates function.

func =  K.function(inputs = [state_pl, action_pl, Q_pl, V_pl], outputs = grads + [temp])


#get batch
S2,A2,R2,S12,D = agent.get_batch()
Q2 = agent.criticQ.model.predict([S2,A2])
V2 = agent.criticV.target_model.predict(S12)
#Q2 = np.array([Q2.flatten()])
#V2 = np.array([V2.flatten()])

grads = func([S2,A2,Q2,V2])
grads,temp = grads[:-1], grads[-1] 

### Roughwork

In [65]:
class TreeNode(object):
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
        
        
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left  = TreeNode(4)
root.left.right = TreeNode(5)
root.right.left = TreeNode(6)
root.right.right = TreeNode(7)

In [66]:
def dfs(root,level,res):
    if root is None:
        return
    else:
        if len(res) < level + 1:
            res.append([])
        res[level].append(root.val)
        dfs(root.left,level+1,res)
        dfs(root.right,level+1,res)
    return res

res = []
res = dfs(root, 0, res)
print res
[x[0] for x in res]

[[1], [2, 3], [4, 5, 6, 7]]


[1, 2, 4]

In [78]:
def invertTree(root):
    """
    :type root: TreeNode
    :rtype: TreeNode
    """
    if not root:
        return
    tmp=root.right
    root.right=invertTree(root.left)
    root.left=invertTree(tmp)
    return root



def invertTree1(root):
    if root:
        root.right, root.left = invertTree(root.left), invertTree(root.right)
    return root


root1 = invertTree(root)
res = []
res = dfs(root, 0, res)
res

[[1], [2, 3], [4, 5, 6, 7]]

In [136]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left  = TreeNode(4)
root.left.right = TreeNode(5)
root.right.left = TreeNode(6)

def dfs(root,level,res):
    if root is None:
        return None
    else:
        if len(res) < level + 1:
            res.append([])
        res[level].append(root.val)
        dfs(root.left,level+1,res)
        dfs(root.right,level+1,res)
    return  res

res = []
dfs(root,0,res)

[[1], [2, 3], [4, 5, 6]]

In [114]:
class Solution(object):
    def binaryTreePaths(self, root):
        """
        :type root: TreeNode
        :rtype: List[str]
        """
        
        res = []
        if root is None:
            return []
        if root.left is None and root.right is None:
            return [str(root.val)]
        paths = [str(root.val) + '->' + path for path in binaryTreePaths(root.left)]
        paths += [str(root.val) + '->' + path for path in binaryTreePaths(root.right)]
        return paths
    
sol = Solution()
sol.binaryTreePaths(root)

['1->2->4', '1->2->5', '1->3->6']

In [120]:
class Solution1(object):
    def binaryTreePaths(self, root):
        """
        :type root: TreeNode
        :rtype: List[str]
        """
        
        res = []
        if root is None:
            return []
        if root.left is None and root.right is None:
            return [str(root.val)]
        paths = [str(root.val) + '->' + path for path in binaryTreePaths(root.left)]
        return paths
    
sol = Solution1()
sol.binaryTreePaths(root)

['1->2->4', '1->2->5']

In [147]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left  = TreeNode(4)
root.left.right = TreeNode(5)
root.right.right = TreeNode(7)


def dfs(root,level,res):
    if root is None:
        return None
    else:
        if len(res) < level + 1:
            res.append([])
        res[level].append(root.val)
        
        if root.left is None and root.right is not None:
            res[level+1].append(None)
        
        dfs(root.left, level+1, res)
        dfs(root.right, level+1, res)
    return max( [len(i) for i in res] )

res = []
res = dfs(root, 0, res)
res

4

In [156]:
def depth(root):
    d = 0
    if root is None:
        return 0
    else:
        d = 1 + max([depth(root.left),depth(root.right)])
    return d
                    
    
def maxPathLength(root):
    dLeft = depth(root.left)
    dRight = depth(root.right)
    return dLeft + dRight 

maxPathLength(root)

4

In [ ]:
class Solution(object):
    def diameterOfBinaryTree(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        self.ans = 0
        
        def depth(p):
            if not p: return 0
            left, right = depth(p.left), depth(p.right)
            self.ans = max(self.ans, left+right)
            return 1 + max(left, right)
            
        depth(root)
        return self.ans